# BS Optimization

- This script will take a fake BS and perform an optimization to determine highest profit generating balances
- After the first successful run, I will layer on additional complexity in terms of contraints

In [57]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [58]:
# Define objective function
# Only used in the version of that uses the 'minimize' function

# def fun(x):
    
#     sva = x * nsi
    
#     return -sva

In [59]:
# Define starting balances
# Not need for linprog implementation

# x0 = np.array(df['start'])

# x0.reshape(-1, 1)

In [60]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=26)

In [61]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,srwa_min
Index,,,,,,,,,
1,prime_auto,10000,5000,-3000,80,1,0.2,1.0,0.11
2,subprime_auto,2000,2000,-500,150,1,0.6,1.0,0.11
3,mtg_30_fixed,40000,20000,-10000,70,1,0.3,0.5,0.11
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0.11
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0.11


In [62]:
# Define objective function coefficients
# We are 'minimizing' the spread so will make this negative
# Dividing by 10,000 to convert bps to spread (will not affect optimization)

c = -np.array(df['spread'] / 10000)

## Inequality Constraint

- The inequality constraint for our optimization is the the SRWA % * 11% * the balance - equity needs to be greater than or equal to 0.
- Since it is greater than, for purposes of Scipy notation, we need to flip the sign

In [63]:
# Define the CET1 minimum

CET1_min = 0.11 

In [64]:
# Multiply the RWA % by the RWA minimum

A_ineq = [list((df['s_rwa'] * df['srwa_min']))]

# Define the other side of the inequality equation (we want to be at least at the minimum, so this is 0)

b_ineq = [0]

In [65]:
A_ineq

[[0.11,
  0.11,
  0.055,
  0.055,
  0.055,
  0.055,
  0.055,
  0.11,
  0.11,
  0.132,
  0.11,
  0.1265,
  0.11,
  0.060500000000000005,
  0.066,
  0.055,
  0.077,
  0.0055000000000000005,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0]]

## Equality Constraint

- This constraint says that Assets - Liabilities must equal to 0.
- I've pre-programmed this to an extent by including a assets (+100%) and liability (-100%) weight in the inputs.

In [66]:
# Create Asset and Liability equality constraint.

A_eq = [list(df['A_L'])]

In [67]:
# Equality constraint vector defined below. This is saying that assets must equal liabilities on the B/S

a_l_constraint = 0

b_eq = [a_l_constraint]

In [68]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [69]:
# Define bounds in format needed for linprog

bounds = list(df['bounds'])

## Optimize B/S

In [70]:
res = spo.linprog(c, A_ub = A_ineq, b_ub = b_ineq, A_eq = A_eq, b_eq = b_eq, bounds = bounds, method='revised simplex')

## Results

In [71]:
# This shows that a solution was found

res.success

True

In [72]:
# Show the output of the objective function

res.fun

-15042.698124999999

In [73]:
# Compare to starting profitability

start_profitability = -sum(df['start'] * df['spread'])

(res.fun - start_profitability) / start_profitability

# Profitability doubled!

-0.9998583950096488

In [74]:
# Show ending balances

res.x

array([  7000.  ,   4000.  ,  60000.  ,  23000.  ,   7500.  ,  30000.  ,
        13500.  ,  70000.  ,  17500.  ,  15000.  ,  50000.  ,  28000.  ,
        65000.  , 105000.  ,  75000.  ,  95000.  ,  34000.  ,   1500.  ,
       235000.  ,  85000.  , 130000.  ,  28969.25, 125500.  ,  38000.  ,
         3000.  ,  55530.75])

In [75]:
# Append ending balance to our data and compare growth vs. shrink

df['optimal_balance'] = res.x

df['balance_change'] = df['optimal_balance'] - df['start']

In [76]:
balance_results = df[['Product', 'balance_change']]

In [77]:
# Aha, this shows that something is wrong in the formula of our optimization.

balance_results

,Product,balance_change
Index,,
1,prime_auto,-3000.00
2,subprime_auto,2000.00
3,mtg_30_fixed,20000.00
4,mtg_15_fixed,8000.00
5,mtg_7_fixed,2500.00
6,mtg_15_arm,10000.00
7,mtg_7_arm,3500.00
8,consumer_card,20000.00
9,business_card,2500.00


In [78]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,srwa_min,u_bound,l_bound,bounds,optimal_balance,balance_change
Index,,,,,,,,,,,,,,
1,prime_auto,10000,5000,-3000,80,1,0.2,1.0,0.11,15000,7000,"(7000, 15000)",7000.0,-3000.0
2,subprime_auto,2000,2000,-500,150,1,0.6,1.0,0.11,4000,1500,"(1500, 4000)",4000.0,2000.0
3,mtg_30_fixed,40000,20000,-10000,70,1,0.3,0.5,0.11,60000,30000,"(30000, 60000)",60000.0,20000.0
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0.11,23000,12000,"(12000, 23000)",23000.0,8000.0
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0.11,7500,3000,"(3000, 7500)",7500.0,2500.0


In [79]:
# Export updated results to Excel

df.to_excel('./results/results.xlsx')